In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
train=pd.read_csv('train.csv')
print train.shape
train.head()

(114321, 133)


,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [19]:
train['v125'].isnull().value_counts()[False]

114244

In [22]:
#there's quite a bit of missing data in the training set
def checkmissing(x):
    for a in x.columns:
        num=x[a].notnull().value_counts()[True]
        print a,(114321-num)*100/114321.0
        print "*****************************"
checkmissing(train)

ID 0.0
*****************************
target 0.0
*****************************
v1 43.589541729
*****************************
v2 43.5580514516
*****************************
v3 3.02394135811
*****************************
v4 43.5580514516
*****************************
v5 42.532867977
*****************************
v6 43.589541729
*****************************
v7 43.589541729
*****************************
v8 42.5284943274
*****************************
v9 43.6061615976
*****************************
v10 0.0734773138793
*****************************
v11 43.5930406487
*****************************
v12 0.0752267737336
*****************************
v13 43.589541729
*****************************
v14 0.00349891970854
*****************************
v15 43.5930406487
*****************************
v16 43.6446497144
*****************************
v17 43.5580514516
*****************************
v18 43.589541729
*****************************
v19 43.5991637582
*****************************
v20 43.5965395684


In [24]:
#The v22 column has over 18000 categories. My poor computer can't handle that. 
train['v22'].unique().shape

(18211,)

In [26]:
# data is imbalanced
train['target'].value_counts()

1    87021
0    27300
Name: target, dtype: int64

In [40]:
#dummy code the missing data
mss_train=train[train.columns[2:]].isnull().astype('int')
mss_train.columns = [a+'_na' for a in train.columns[2:]]

# remove low occuring categories from v22 column. Set aside the target column
target= train['target']
v22 = train['v22']
train.drop(['target','ID'],axis=1,inplace=True)
goodnum=train['v22'].value_counts()[(train['v22'].value_counts()>150)]
train['v22']=train['v22'].apply(lambda x : x if x in goodnum.index else 'JUNK')

#get the list of columns that are categorical
cate_col=[a for a in train.columns if train[a].dtypes=='object']
fummy =[a for a in train.columns if a not in cate_col]
nummy= train[fummy]
catty = train[cate_col]


#dummy code categorical columns and sort columns, 
# which will be important to make sure that the test data columns are aligned. 
catty=pd.get_dummies(catty)
sort_cols = np.sort(catty.columns)
catty=catty[sort_cols]

In [41]:
#On the numerical columns, impute missing data for now with the median value.
imp=Imputer(axis=1,verbose=1,strategy='median')
tainy=imp.fit_transform(nummy)

#scale numerical columns
scaler = StandardScaler().fit(tainy)
tainy =scaler.transform(tainy)
trn=np.hstack((tainy,catty.values,mss_train.values))

In [243]:
def checkcate(x):
    for a in cate_col:
        print pd.crosstab(target,x[a],aggfunc='counts')
        print "************************"
checkcate(train)

v3        A   B      C
target                
0        10   6  26607
1       217  47  83977
************************
v22     AAPP  ABF  ABOF  ACWE  ADDF  ADMI  ADMP  AFOZ  AFYU  AGDF ...    TG  \
target                                                            ...         
0         41   43    69    50    63    50    38    41    39   505 ...    45   
1        123  123   135   140   149   106   134   111   120  1881 ...   121   

v22     TVR  UAG  VVI  VZF  WNI  WRI  YEP   YGJ  YOD  
target                                                
0        60   31   60   73   44   41   61   524   77  
1       122  138  151  223  200  166  163  1595  226  

[2 rows x 44 columns]
************************
v24        A     B      C      D      E
target                                 
0       1015  1929   4183   6731  13442
1       2774  6221  16689  19602  41735
************************
v30        A    B      C     D     E     F     G
target                                          
0        482   

In [39]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [51]:
ts=pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')

In [154]:
# the categories between the test and train data are not identical
[(a,len(train[a].value_counts()),len(test[a].value_counts())) for a in cate_col]

[('v3', 3, 3),
 ('v22', 44, 41),
 ('v24', 5, 5),
 ('v30', 7, 7),
 ('v31', 3, 3),
 ('v47', 10, 9),
 ('v52', 12, 12),
 ('v56', 122, 116),
 ('v66', 3, 3),
 ('v71', 9, 9),
 ('v74', 3, 3),
 ('v75', 4, 4),
 ('v79', 18, 17),
 ('v91', 7, 7),
 ('v107', 7, 7),
 ('v110', 3, 3),
 ('v112', 22, 22),
 ('v113', 36, 36),
 ('v125', 90, 90)]

In [53]:
#preprocessing for the test set data
def prepoc(t):
    t['v22']=t['v22'].apply(lambda x : x if x in goodnum.index else 'JUNK')

    nummy2= t[fummy]
    catty2 = t[cate_col]
    catty2=pd.get_dummies(catty2)   
    
    # there are some categories in test set data that don't exist in the training set
    # and vice versa. We have to sort that out to make sure the number and order of columns
    # in the test and train are identical. 
    for a in catty.columns:
        if a not in catty2.columns:
            catty2[str(a)] = 0
    for a in catty2.columns:
        if a not in catty.columns:
            catty2.drop(a,axis=1,inplace=True)
    sor_col = np.sort(catty2.columns)
    catty2 = catty2[sor_col]
    print "woohoo"
    
    mss_traint=t[train.columns[2:]].isnull().astype('int')
    mss_traint.columns = [a+'_na' for a in train.columns[2:]]

    tainy2=imp.fit_transform(nummy2)
    scaler = StandardScaler().fit(tainy2)
    tainy =scaler.transform(tainy2)
    tst=np.hstack((tainy2,catty2.values,mss_traint.values))
    return tst,catty2

In [54]:
t,catty2=prepoc(test)

woohoo


In [44]:
lr = LogisticRegression(class_weight='auto')
#gbc = GradientBoostingClassifier(n_estimators=500)
#abc = AdaBoostClassifier()
etc = ExtraTreesClassifier(n_estimators=50,class_weight='auto')
gnb = GaussianNB()
svc = LinearSVC()
rfc = RandomForestClassifier(n_estimators=50,class_weight='auto')
bag =[etc,rfc,lr,gnb,svc]


print "Training classifiers"
for a in bag:
    print a
    print cross_val_score(a,trn,target).mean()
    print cross_val_score(a,trn,target,scoring='average_precision').mean()
    print cross_val_score(a,trn,target,scoring='roc_auc').mean()
    a.fit(trn,target)
    print "********************************"

Training classifiers
ExtraTreesClassifier(bootstrap=False, class_weight='auto', criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
0.772937605514
0.889578590378
0.727388263361
********************************
RandomForestClassifier(bootstrap=True, class_weight='auto', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.774372162595
0.885382775914
0.723254306684
********************************
LogisticRegression(C=1.0, class_weight='auto', dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100

In [ ]:
#try some good old dimensionality reduction
from sklearn.decomposition import PCA, IncrementalPCA
n_components = 50
ipca = IncrementalPCA(n_components=n_components, batch_size=5000)
X_ipca = ipca.fit_transform(trn)

In [50]:
lr = LogisticRegression(class_weight='balanced')
#gbc = GradientBoostingClassifier(n_estimators=500)
#abc = AdaBoostClassifier()
etc = ExtraTreesClassifier(n_estimators=100,class_weight='balanced',criterion='entropy')
gnb = GaussianNB()
svc = LinearSVC()
rfc = RandomForestClassifier(n_estimators=100,class_weight='balanced',criterion='entropy')
bag =[etc,rfc,lr,gnb,svc]
#test classifiers again
for a in bag:
    print a
    print cross_val_score(a,X_ipca,target).mean()
    print cross_val_score(a,X_ipca,target,scoring='roc_auc').mean()
    print "********************************"

#pca = PCA(n_components=n_components)
#X_pca = pca.fit_transform(trn)

ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='entropy', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
0.770392141426
0.726402302665
********************************
RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.771214387558
0.730302421633
********************************
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', 

In [306]:
#use random forests to pick out important features. Train classifiers based on 
# these columns
feat_import=np.where(rfc.feature_importances_ >0.01)

trn_ft=trn.T[feat_import].T
for a in bag:
    print cross_val_score(a,trn_ft,target).mean()
    print cross_val_score(a,trn_ft,target,scoring='average_precision').mean()
    print cross_val_score(a,trn_ft,target,scoring='roc_auc').mean()
    print "********************************"

rand_gp =[a.predict(trn_ft) for a in rand_ext]

In [ ]:
#Do another training session but using imputation with mean values
#On the numerical columns, impute missing data for now with the median value.
imp=Imputer(axis=1,verbose=1)
tainy=imp.fit_transform(nummy)

#scale numerical columns
scaler = StandardScaler().fit(tainy)
tainy =scaler.transform(tainy)
trn=np.hstack((tainy,catty.values,mss_train.values))

print "Training classifiers"
for a in bag:
    print a
    print cross_val_score(a,trn,target).mean()
    print cross_val_score(a,trn,target,scoring='average_precision').mean()
    print cross_val_score(a,trn,target,scoring='roc_auc').mean()
    a.fit(trn,target)
    print "********************************"
